In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Deep Learning/Object Detection/Number-Plate

/content/drive/MyDrive/Deep Learning/Object Detection/Number-Plate


In [ ]:
pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=1a6f9696cb1f4820f35a4de3be55c2acd98fcef53d6a04a543e6d55b3265c12a
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


In [ ]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.8/699.8 kB 8.4 MB/s eta 0:00:00


In [ ]:
pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 29.5 MB/s eta 0:00:00


In [ ]:
import cv2
import pandas as pd
import numpy as np
from google.colab.patches import cv2_imshow
from sort.sort import *

In [ ]:
pwd

'/content/drive/MyDrive/Deep Learning/Object Detection/Number-Plate'

In [ ]:
#!git clone https://github.com/abewley/sort.git

In [ ]:
#pip install ultralytics
from ultralytics import YOLO
from util import *

In [ ]:
result={}

coco_model = YOLO('yolov8n.pt')
license_plate_detector = YOLO('license_plate_detector.pt')

mot_tracker = Sort()
cap = cv2.VideoCapture('video.mp4')
total_frame=cap.get(cv2.CAP_PROP_FRAME_COUNT)

vehicles = [2, 3, 5, 7]

#read frame
frame_no=-1
ret=True
while ret:
    frame_no+=1
    ret,frame=cap.read()
    if ret:

        fps = cap.get(cv2.CAP_PROP_FPS)

        result[frame_no]={}
        results=coco_model(frame)
        vehicle_detect=[]
        for detection in results[0].boxes.data:
            x1,y1,x2,y2,score,class_id=detection.tolist()

            if int(class_id) in vehicles:
                vehicle_detect.append([x1,y1,x2,y2,score])
                cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),(0,255,0),2 )

        # Track Vehicles
        track_ids = mot_tracker.update(np.asarray(vehicle_detect))

        # detect license plates
        license_plates = license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)
            cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),(0,255,0),2 )

            if car_id!=-1:
              #crop license plate
                license_plate_crop=frame[int(y1):int(y2),int(x1):int(x2):]

                # Process License Plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                # Read License Plate
                license_plate_text, license_plate_text_score=read_license_plate(license_plate_crop_thresh)
                cv2.putText(frame,license_plate_text , (int(x1), int(y1)), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)


              #license_plate_text,license_plate_text_score='NA13NRU',0.87
              if license_plate_text is not None:
                result[frame_no][car_id]={'car':{'bbox':[xcar1,ycar1,xcar2,ycar2]},
                                              'license_plate':{'bbox':[x1,y1,x2,y2],
                                                               'text':license_plate_text,
                                                               'bbox_score':score,
                                                               'text_score':license_plate_text_score}
                                              }

        if frame_no%100==0:
            print('Frame Parse {}/{}'.format(frame_no,total_frame))

        display_width = 1200  # Replace with your desired width
        display_height = 600  # Replace with your desired height

        # Create a window with the specified dimensions
        cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('frame', display_width, display_height)
        resized_frame = cv2.resize(frame, (display_width, display_height))
        cv2.putText(resized_frame, f'FPS: {fps:.2f}', (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1, cv2.LINE_AA)

        cv2_imshow('frame',resized_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    #cv2_imshow(frame)

In [ ]:
save_result(result)

In [ ]:
def save_result(result):

  df_final=pd.DataFrame()
  for frame_no in result.keys():


    for car_id in result[frame_no].keys():

      df1=pd.DataFrame({'bbox':[result[frame_no][car_id]['car']['bbox']]})
      df2=pd.DataFrame({'license_plate':[result[frame_no][car_id]['license_plate']['bbox']],
                    'text':result[frame_no][car_id]['license_plate']['text'],
                    'bbox_score':result[frame_no][car_id]['license_plate']['bbox_score'],
                    'text_score':result[frame_no][car_id]['license_plate']['text_score']})
      df=pd.concat([df1,df2],axis=1)
      df.insert(0, 'frame', frame_no)
      df.insert(1, 'car_id', car_id)

      df_final=pd.concat([df_final,df],axis=0)
  return df_final.to_csv('result.csv',index=False)

,frame,car_id,bbox,license_plate,text,bbox_score,text_score
0,0,157.0,"[2195.68017578125, 1157.049072265625, 2786.025...","[2407.13818359375, 1551.4912109375, 2576.46093...",NA13NRU,0.601363,0.87
0,0,155.0,"[752.2139282226562, 1369.3121337890625, 1429.3...","[974.8357543945312, 1753.404052734375, 1199.59...",NA13NRU,0.566925,0.87


In [ ]:
result[frame_no][car_id]['license_plate']

{'bbox': [2407.13818359375, 1551.4912109375, 2576.4609375, 1638.7039794921875],
 'text': 'NA13NRU',
 'bbox_score': 0.6013634204864502,
 'text_score': 0.87}

In [ ]:
pd.DataFrame({'license_plate':[result[frame_no][car_id]['license_plate']['bbox']],
              'text':result[frame_no][car_id]['license_plate']['text'],
              'bbox_score':result[frame_no][car_id]['license_plate']['bbox_score'],
              'text_score':result[frame_no][car_id]['license_plate']['text_score']})

,license_plate,text,bbox_score,text_score
0,"[2407.13818359375, 1551.4912109375, 2576.46093...",NA13NRU,0.601363,0.87


In [ ]:

df

,bbox,license_plate,text,bbox_score,text_score,frame,car_id
0,"[2195.68017578125, 1157.049072265625, 2786.025...","[2407.13818359375, 1551.4912109375, 2576.46093...",NA13NRU,0.601363,0.87,0,157.0


In [ ]:
cap.get(cv2.CAP_PROP_FRAME_COUNT)

1800.0

In [ ]:
def write_csv(result,output_path):

  with open(output_path,'w') as f :
    f.write('{},{},{},{},{},{},{}\n'.format('frame_no','car_id','car_bbox',
                                            'license_plate_bbox','license_plate_bbox_score',
                                            'license_number','license_number_score'))

    for frame_no in results.key():

False

In [ ]:
digits

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
dict_int_to_char

{'0': 'O', '1': 'I', '3': 'J', '4': 'A', '6': 'G', '5': 'S'}